In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow
import sklearn


# local imports
from prepare import *
from evaluate import *

In [2]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from catboost import CatBoostRegressor
    
# Try out balanced classes
reduced_train = pd.read_csv('reduce_train.csv')

#reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv('reduce_test.csv')
reduced_train.shape, reduced_test.shape

categoricals = ['session_title']
cols_to_drop = ['game_session', 'installation_id', 'accuracy_group']


C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\imp.py:343: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return _load(spec)


In [3]:
mlflow.set_experiment('Big-Three')

In [4]:
lgbm = LGBMRegressor()
rfr = RandomForestRegressor()
cbr = CatBoostRegressor(
    loss_function="RMSE",
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42,
)
gbr = GradientBoostingRegressor()

In [5]:
lgbm = cv_reg(lgbm, reduced_train)

1/7 Fold start| QWK=0.5758965435475357
2/7 Fold start| QWK=0.550756444679084
3/7 Fold start| QWK=0.5601402554969841
4/7 Fold start| QWK=0.5550962255515917
5/7 Fold start| QWK=0.5227226572040957
6/7 Fold start| QWK=0.5377995289001236
7/7 Fold start| QWK=0.5454295357123363


In [6]:
cbr = cv_reg(cbr, reduced_train)

0:	learn: 1.2388741	total: 104ms	remaining: 3m 27s
1:	learn: 1.2240404	total: 147ms	remaining: 2m 26s
2:	learn: 1.2098334	total: 192ms	remaining: 2m 8s
3:	learn: 1.1956890	total: 234ms	remaining: 1m 56s
4:	learn: 1.1821780	total: 277ms	remaining: 1m 50s
5:	learn: 1.1699729	total: 328ms	remaining: 1m 48s
6:	learn: 1.1587198	total: 371ms	remaining: 1m 45s
7:	learn: 1.1488837	total: 414ms	remaining: 1m 43s
8:	learn: 1.1395892	total: 453ms	remaining: 1m 40s
9:	learn: 1.1312497	total: 496ms	remaining: 1m 38s
10:	learn: 1.1228996	total: 543ms	remaining: 1m 38s
11:	learn: 1.1160376	total: 588ms	remaining: 1m 37s
12:	learn: 1.1091516	total: 630ms	remaining: 1m 36s
13:	learn: 1.1033091	total: 672ms	remaining: 1m 35s
14:	learn: 1.0970247	total: 715ms	remaining: 1m 34s
15:	learn: 1.0920114	total: 756ms	remaining: 1m 33s
16:	learn: 1.0867676	total: 804ms	remaining: 1m 33s
17:	learn: 1.0824540	total: 843ms	remaining: 1m 32s
18:	learn: 1.0779017	total: 886ms	remaining: 1m 32s
19:	learn: 1.0740184	to

160:	learn: 0.9706480	total: 7.98s	remaining: 1m 31s
161:	learn: 0.9704899	total: 8.03s	remaining: 1m 31s
162:	learn: 0.9703618	total: 8.07s	remaining: 1m 30s
163:	learn: 0.9703215	total: 8.1s	remaining: 1m 30s
164:	learn: 0.9699634	total: 8.15s	remaining: 1m 30s
165:	learn: 0.9697461	total: 8.19s	remaining: 1m 30s
166:	learn: 0.9695833	total: 8.24s	remaining: 1m 30s
167:	learn: 0.9695326	total: 8.28s	remaining: 1m 30s
168:	learn: 0.9692306	total: 8.32s	remaining: 1m 30s
169:	learn: 0.9690315	total: 8.37s	remaining: 1m 30s
170:	learn: 0.9686875	total: 8.41s	remaining: 1m 29s
171:	learn: 0.9686119	total: 8.46s	remaining: 1m 29s
172:	learn: 0.9683268	total: 8.52s	remaining: 1m 29s
173:	learn: 0.9681555	total: 8.56s	remaining: 1m 29s
174:	learn: 0.9680506	total: 8.6s	remaining: 1m 29s
175:	learn: 0.9675742	total: 8.66s	remaining: 1m 29s
176:	learn: 0.9672978	total: 8.71s	remaining: 1m 29s
177:	learn: 0.9670469	total: 8.75s	remaining: 1m 29s
178:	learn: 0.9667595	total: 8.79s	remaining: 1m

320:	learn: 0.9298187	total: 15.3s	remaining: 1m 20s
321:	learn: 0.9295527	total: 15.4s	remaining: 1m 20s
322:	learn: 0.9293433	total: 15.4s	remaining: 1m 20s
323:	learn: 0.9291173	total: 15.5s	remaining: 1m 20s
324:	learn: 0.9289065	total: 15.5s	remaining: 1m 20s
325:	learn: 0.9286193	total: 15.6s	remaining: 1m 20s
326:	learn: 0.9283915	total: 15.6s	remaining: 1m 19s
327:	learn: 0.9281382	total: 15.7s	remaining: 1m 19s
328:	learn: 0.9280923	total: 15.7s	remaining: 1m 19s
329:	learn: 0.9278516	total: 15.8s	remaining: 1m 19s
330:	learn: 0.9276852	total: 15.8s	remaining: 1m 19s
331:	learn: 0.9273800	total: 15.9s	remaining: 1m 19s
332:	learn: 0.9270092	total: 15.9s	remaining: 1m 19s
333:	learn: 0.9267265	total: 16s	remaining: 1m 19s
334:	learn: 0.9263361	total: 16s	remaining: 1m 19s
335:	learn: 0.9263063	total: 16.1s	remaining: 1m 19s
336:	learn: 0.9260285	total: 16.1s	remaining: 1m 19s
337:	learn: 0.9257739	total: 16.2s	remaining: 1m 19s
338:	learn: 0.9254709	total: 16.2s	remaining: 1m 1

476:	learn: 0.8936870	total: 22.7s	remaining: 1m 12s
477:	learn: 0.8934990	total: 22.8s	remaining: 1m 12s
478:	learn: 0.8932805	total: 22.8s	remaining: 1m 12s
479:	learn: 0.8930813	total: 22.9s	remaining: 1m 12s
480:	learn: 0.8927201	total: 22.9s	remaining: 1m 12s
481:	learn: 0.8925212	total: 23s	remaining: 1m 12s
482:	learn: 0.8925037	total: 23s	remaining: 1m 12s
483:	learn: 0.8922953	total: 23.1s	remaining: 1m 12s
484:	learn: 0.8921205	total: 23.1s	remaining: 1m 12s
485:	learn: 0.8918303	total: 23.2s	remaining: 1m 12s
486:	learn: 0.8915685	total: 23.2s	remaining: 1m 12s
487:	learn: 0.8913969	total: 23.3s	remaining: 1m 12s
488:	learn: 0.8912934	total: 23.3s	remaining: 1m 11s
489:	learn: 0.8912669	total: 23.3s	remaining: 1m 11s
490:	learn: 0.8910717	total: 23.4s	remaining: 1m 11s
491:	learn: 0.8908435	total: 23.4s	remaining: 1m 11s
492:	learn: 0.8906119	total: 23.5s	remaining: 1m 11s
493:	learn: 0.8903714	total: 23.5s	remaining: 1m 11s
494:	learn: 0.8902493	total: 23.6s	remaining: 1m 1

634:	learn: 0.8653148	total: 30.3s	remaining: 1m 5s
635:	learn: 0.8650967	total: 30.3s	remaining: 1m 5s
636:	learn: 0.8649785	total: 30.4s	remaining: 1m 5s
637:	learn: 0.8648120	total: 30.4s	remaining: 1m 4s
638:	learn: 0.8646296	total: 30.5s	remaining: 1m 4s
639:	learn: 0.8644225	total: 30.6s	remaining: 1m 4s
640:	learn: 0.8643285	total: 30.6s	remaining: 1m 4s
641:	learn: 0.8641956	total: 30.7s	remaining: 1m 4s
642:	learn: 0.8639118	total: 30.7s	remaining: 1m 4s
643:	learn: 0.8637288	total: 30.8s	remaining: 1m 4s
644:	learn: 0.8634893	total: 30.8s	remaining: 1m 4s
645:	learn: 0.8634013	total: 30.9s	remaining: 1m 4s
646:	learn: 0.8632790	total: 30.9s	remaining: 1m 4s
647:	learn: 0.8631657	total: 31s	remaining: 1m 4s
648:	learn: 0.8629477	total: 31s	remaining: 1m 4s
649:	learn: 0.8627242	total: 31.1s	remaining: 1m 4s
650:	learn: 0.8626211	total: 31.1s	remaining: 1m 4s
651:	learn: 0.8625382	total: 31.2s	remaining: 1m 4s
652:	learn: 0.8623500	total: 31.2s	remaining: 1m 4s
653:	learn: 0.86

...

1584:	learn: 0.7423120	total: 1m 18s	remaining: 20.5s
1585:	learn: 0.7421832	total: 1m 18s	remaining: 20.5s
1586:	learn: 0.7420710	total: 1m 18s	remaining: 20.4s
1587:	learn: 0.7420693	total: 1m 18s	remaining: 20.4s
1588:	learn: 0.7419102	total: 1m 18s	remaining: 20.3s
1589:	learn: 0.7418992	total: 1m 18s	remaining: 20.3s
1590:	learn: 0.7417556	total: 1m 18s	remaining: 20.2s
1591:	learn: 0.7416333	total: 1m 18s	remaining: 20.2s
1592:	learn: 0.7415836	total: 1m 18s	remaining: 20.1s
1593:	learn: 0.7414432	total: 1m 18s	remaining: 20.1s
1594:	learn: 0.7413000	total: 1m 18s	remaining: 20s
1595:	learn: 0.7412188	total: 1m 18s	remaining: 20s
1596:	learn: 0.7411558	total: 1m 19s	remaining: 19.9s
1597:	learn: 0.7410311	total: 1m 19s	remaining: 19.9s
1598:	learn: 0.7408929	total: 1m 19s	remaining: 19.8s
1599:	learn: 0.7407505	total: 1m 19s	remaining: 19.8s
1600:	learn: 0.7406995	total: 1m 19s	remaining: 19.8s
1601:	learn: 0.7406020	total: 1m 19s	remaining: 19.7s
1602:	learn: 0.7404950	total: 1m

1738:	learn: 0.7258271	total: 1m 25s	remaining: 12.8s
1739:	learn: 0.7257084	total: 1m 25s	remaining: 12.8s
1740:	learn: 0.7256534	total: 1m 25s	remaining: 12.7s
1741:	learn: 0.7255390	total: 1m 25s	remaining: 12.7s
1742:	learn: 0.7254204	total: 1m 25s	remaining: 12.6s
1743:	learn: 0.7253177	total: 1m 25s	remaining: 12.6s
1744:	learn: 0.7252075	total: 1m 25s	remaining: 12.5s
1745:	learn: 0.7250227	total: 1m 25s	remaining: 12.5s
1746:	learn: 0.7249758	total: 1m 25s	remaining: 12.4s
1747:	learn: 0.7248341	total: 1m 25s	remaining: 12.4s
1748:	learn: 0.7247470	total: 1m 25s	remaining: 12.3s
1749:	learn: 0.7246609	total: 1m 25s	remaining: 12.3s
1750:	learn: 0.7245974	total: 1m 26s	remaining: 12.2s
1751:	learn: 0.7244402	total: 1m 26s	remaining: 12.2s
1752:	learn: 0.7242856	total: 1m 26s	remaining: 12.1s
1753:	learn: 0.7241430	total: 1m 26s	remaining: 12.1s
1754:	learn: 0.7240603	total: 1m 26s	remaining: 12s
1755:	learn: 0.7239621	total: 1m 26s	remaining: 12s
1756:	learn: 0.7238480	total: 1m

1892:	learn: 0.7097394	total: 1m 32s	remaining: 5.22s
1893:	learn: 0.7096873	total: 1m 32s	remaining: 5.17s
1894:	learn: 0.7095931	total: 1m 32s	remaining: 5.12s
1895:	learn: 0.7095384	total: 1m 32s	remaining: 5.07s
1896:	learn: 0.7094196	total: 1m 32s	remaining: 5.02s
1897:	learn: 0.7093159	total: 1m 32s	remaining: 4.97s
1898:	learn: 0.7092915	total: 1m 32s	remaining: 4.92s
1899:	learn: 0.7092139	total: 1m 32s	remaining: 4.87s
1900:	learn: 0.7091532	total: 1m 32s	remaining: 4.83s
1901:	learn: 0.7090814	total: 1m 32s	remaining: 4.78s
1902:	learn: 0.7089490	total: 1m 32s	remaining: 4.73s
1903:	learn: 0.7088602	total: 1m 32s	remaining: 4.68s
1904:	learn: 0.7087739	total: 1m 32s	remaining: 4.63s
1905:	learn: 0.7086355	total: 1m 32s	remaining: 4.58s
1906:	learn: 0.7085575	total: 1m 32s	remaining: 4.53s
1907:	learn: 0.7084724	total: 1m 33s	remaining: 4.49s
1908:	learn: 0.7084703	total: 1m 33s	remaining: 4.44s
1909:	learn: 0.7084553	total: 1m 33s	remaining: 4.39s
1910:	learn: 0.7083459	total

46:	learn: 1.0220628	total: 2.1s	remaining: 1m 27s
47:	learn: 1.0211387	total: 2.15s	remaining: 1m 27s
48:	learn: 1.0204368	total: 2.2s	remaining: 1m 27s
49:	learn: 1.0194703	total: 2.25s	remaining: 1m 27s
50:	learn: 1.0184775	total: 2.3s	remaining: 1m 27s
51:	learn: 1.0176677	total: 2.37s	remaining: 1m 28s
52:	learn: 1.0167243	total: 2.42s	remaining: 1m 29s
53:	learn: 1.0157765	total: 2.49s	remaining: 1m 29s
54:	learn: 1.0147208	total: 2.54s	remaining: 1m 29s
55:	learn: 1.0136500	total: 2.59s	remaining: 1m 29s
56:	learn: 1.0130574	total: 2.63s	remaining: 1m 29s
57:	learn: 1.0121323	total: 2.68s	remaining: 1m 29s
58:	learn: 1.0116857	total: 2.73s	remaining: 1m 29s
59:	learn: 1.0109808	total: 2.77s	remaining: 1m 29s
60:	learn: 1.0101705	total: 2.82s	remaining: 1m 29s
61:	learn: 1.0094216	total: 2.87s	remaining: 1m 29s
62:	learn: 1.0084158	total: 2.92s	remaining: 1m 29s
63:	learn: 1.0077267	total: 2.97s	remaining: 1m 29s
64:	learn: 1.0070765	total: 3.02s	remaining: 1m 29s
65:	learn: 1.00

KeyboardInterrupt: 